existing recsys & different dataset & own analysis

recommeder system : content based filtering
https://github.com/lsjsj92/recommender_system_with_Python/blob/master/002.%20recommender%20system%20basic%20with%20Python%20-%201%20content%20based%20filtering.ipynb

dataset : anime recommendations database
https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database

In [1]:
# Setting for preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('./anime.csv')

FileNotFoundError: [Errno 2] No such file or directory: './anime.csv'

In [ ]:
data.head(2)

In [ ]:
data.shape

There's a issue on rating because number of ratings effects on rating.(usually inverse proportion.)
So, We have to use 'weighted rating(WR)'

weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C where:

R = average for the movie (mean) = (Rating)
v = number of votes for the movie = (votes)
m = minimum votes required to be listed in the Top 123 (currently 12294)
C = the mean vote across the whole report (currently )

reference : https://www.quora.com/How-does-IMDbs-rating-system-work

In [ ]:
tmp_m = data['members'].quantile(0.99)
tmp_m

In [ ]:
tmp_data = data.copy().loc[data['members'] >= tmp_m]
tmp_data.shape

In [ ]:
del tmp_data

m = data['members'].quantile(0.99)
data = data.loc[data['members'] >= m]

In [3]:
data.head()

NameError: name 'data' is not defined

In [4]:
C = data['rating'].mean()

NameError: name 'data' is not defined

In [454]:
print(C)
print(m)

8.17349593495935
276774.29999999923


In [455]:
def weighted_rating(x, m=m, C=C):
    v = x['members']
    R = x['rating']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [456]:
for index, row in data.iterrows():
    data.loc[index, 'WR'] = weighted_rating(row)

In [457]:
data.head(5)

,anime_id,name,genre,type,episodes,rating,members,WR
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,8.979072
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,8.879783
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855,8.753221
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749,8.725503
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201,9.04,336376,8.648863


In [458]:
data.shape

(123, 8)

In [459]:
data.genre.head(2)

1    Action, Adventure, Drama, Fantasy, Magic, Mili...
3                                     Sci-Fi, Thriller
Name: genre, dtype: object

In [460]:
#vectorize each words of each datas's genre
count_vector = CountVectorizer(ngram_range=(1,9))

In [461]:
c_vector_genre = count_vector.fit_transform(data['genre'])

In [462]:
c_vector_genre.shape

(123, 1062)

In [463]:
#measure cosine similarity of animes by their genre
genre_cos_sim = cosine_similarity(c_vector_genre, c_vector_genre).argsort()[:, ::-1]

In [464]:
genre_cos_sim.shape

(123, 123)

In [465]:
def get_recommend_anime_list(df, anime_name, top=10):
    
    #get info of anime inputted
    target_anime_index = df[df['name'] == anime_name].index.values
    
    #use datas only get top10 cosine similarity
    sim_index = genre_cos_sim[target_anime_index, :top].reshape(-1)
    
    #except itself
    sim_index = sim_index[sim_index != target_anime_index]
    
    #validation of its index
    sim_index = [idx for idx in sim_index if idx in df.index]
    
    #sort recommendation by WR
    result = df.loc[sim_index].sort_values('WR', ascending=False)[:10]
    return result

In [466]:
get_recommend_anime_list(data, anime_name='Fullmetal Alchemist: Brotherhood')

,anime_id,name,genre,type,episodes,rating,members,WR
36,11741,Fate/Zero 2nd Season,"Action, Fantasy, Supernatural, Thriller",TV,12,8.73,340973,8.480665
118,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291,8.376645
104,205,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26,8.50,390076,8.364485


In [467]:
get_recommend_anime_list(data, anime_name='Hunter x Hunter (2011)')

,anime_id,name,genre,type,episodes,rating,members,WR
40,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917,8.594953
36,11741,Fate/Zero 2nd Season,"Action, Fantasy, Supernatural, Thriller",TV,12,8.73,340973,8.480665


In [468]:
get_recommend_anime_list(data, anime_name='One Punch Man')

,anime_id,name,genre,type,episodes,rating,members,WR
49,31043,Boku dake ga Inai Machi,"Mystery, Psychological, Seinen, Supernatural",TV,12,8.65,402381,8.455812
96,9756,Mahou Shoujo Madoka★Magica,"Drama, Magic, Psychological, Thriller",TV,12,8.51,462974,8.384098
118,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291,8.376645
104,205,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26,8.50,390076,8.364485
113,30503,Noragami Aragoto,"Action, Adventure, Shounen, Supernatural",TV,13,8.48,299434,8.332775


<h3>Evaluation of Recommendation</h3>

As a result of examining and evaluating the data on the results, it was also found that the plot of the input animation and the recommended animation was very different.

The reason for this is first presumed to be the bias of animation for commercial purposes, and the association rule confirmed the tendency of certain genres to be selected at the same time when setting the genre of animation.

The second reason was the lack of data on the plot, which could not be reflected in the recommendation. Even if there was, measuring the similarity of the plot would be quite complicated.

<h4>Association Rule Mining</h4>

In [513]:
from mlxtend.preprocessing import TransactionEncoder

#I used data 'members(rating)'s quantile is over than 0.99. and it means they are popular animation.
dataset = data['genre'].str.split(', ')
           
te = TransactionEncoder()

# te_ary = te.fit_transform(dataset) <- Same result
te_ary = te.fit(dataset).transform(dataset)

In [514]:
print(te_ary)

[[ True  True False ... False False False]
 [False False False ... False  True False]
 [ True  True False ... False False False]
 ...
 [ True  True False ... False False False]
 [False False  True ... False False  True]
 [False False False ... False False False]]


In [515]:
print(te.columns_)

['Action', 'Adventure', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Historical', 'Horror', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shounen', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire']


In [516]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [518]:
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)

In [519]:
frequent_itemsets

,support,itemsets
0,0.552846,(Action)
1,0.252033,(Adventure)
2,0.430894,(Comedy)
3,0.292683,(Drama)
4,0.276423,(Fantasy)
5,0.154472,(Mystery)
6,0.138211,(Psychological)
7,0.317073,(Romance)
8,0.300813,(School)
9,0.195122,(Sci-Fi)


In [520]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Adventure),(Action),0.252033,0.552846,0.195122,0.774194,1.400380,0.055787,1.980256,0.382246
1,(Fantasy),(Action),0.276423,0.552846,0.195122,0.705882,1.276817,0.042303,1.520325,0.299625
2,(Sci-Fi),(Action),0.195122,0.552846,0.146341,0.750000,1.356618,0.038469,1.788618,0.326599
3,(Shounen),(Action),0.292683,0.552846,0.219512,0.750000,1.356618,0.057704,1.788618,0.371648
4,(Super Power),(Action),0.162602,0.552846,0.138211,0.850000,1.537500,0.048318,2.981030,0.417476
5,(Adventure),(Fantasy),0.252033,0.276423,0.178862,0.709677,2.567362,0.109194,2.492322,0.816206
6,(School),(Comedy),0.300813,0.430894,0.211382,0.702703,1.630801,0.081764,1.914265,0.553220
7,"(Adventure, Action)",(Fantasy),0.195122,0.276423,0.138211,0.708333,2.562500,0.084275,2.480836,0.757576
8,"(Adventure, Fantasy)",(Action),0.178862,0.552846,0.138211,0.772727,1.397727,0.039328,1.967480,0.346535
9,"(Action, Fantasy)",(Adventure),0.195122,0.252033,0.138211,0.708333,2.810484,0.089034,2.564460,0.800357


So, this association rules make the recommendation by genre more hard.